# Aula 08 - Exercício 07
## Árvores de decisão e Naive-Bayes
### Alunos:
- Alexandre Batistella Bellas, 9763168
- Moisés Botarro Ferraz Silva, 8504135

---
O dataset *Statlog (German Credit Data) Data Set* é um dataset que visa classificar o risco de crédito de pessoas através de um conjunto de atributos. A classificação é feita em risco "Good" (bom - baixo risco) ou risco "Bad" (ruim - alto risco). Ao todo são 24 atributos numéricos e 1 atributo nominal, que representa a classe.

Esse dataset também apresenta o que chamamos de *matriz de custo*. Essa matriz é similar à matriz de confusão para o caso binário, porém, os elementos dessa matriz representam o custo associado com cada tipo de classificação. 

A matriz de custo desse dataset é dada abaixo:

|                  x | Good (Predito) | Bad (Predito) |
|-------------------:|------------:|-----------:|
| **Good (Real)** |           0 |          1 |
|  **Bad (Real)** |           5 |          0 |

A análsie dessa matriz pode ser feita da seguinte maneira: se uma classe real "Good" for predita como "Bad", existe um custo 1 associado. Todavia, se uma classe real "Bad" for predita como "Good", o custo associado é 5. Ou seja, um tipo de erro é melhor (menos custoso) do que o outro. 

O dataset pode ser carregado através do arquivo **data.csv**.


---
### Questão 01. 

Carregue o dataset e separe o conjunto em atributos e classe (X e y).

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')

def partition_data(data):
    x = data[data.columns[:-1]].values
    y = data[data.columns[-1]].values
    return x, y

x, y = partition_data(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,class
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,good
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,bad
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,good
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,good
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,bad


In [2]:
print("# dados:", x.shape[0])
print(x[:5])

# dados: 1000
[[ 1  6  4 12  5  5  3  4  1 67  3  2  1  2  1  0  0  1  0  0  1  0  0  1]
 [ 2 48  2 60  1  3  2  2  1 22  3  1  1  1  1  0  0  1  0  0  1  0  0  1]
 [ 4 12  4 21  1  4  3  3  1 49  3  1  2  1  1  0  0  1  0  0  1  0  1  0]
 [ 1 42  2 79  1  4  3  4  2 45  3  1  2  1  1  0  0  0  0  0  0  0  0  1]
 [ 1 24  3 49  1  3  3  4  4 53  3  2  2  1  1  1  0  1  0  0  0  0  0  1]]


In [3]:
print("# dados:", y.shape[0])
print(y[:5])

# dados: 1000
['good' 'bad' 'good' 'good' 'bad']


---
### Questão 02.
Utilize 10-fold cross-validation estratificado para estimar a acurácia do classificador `sklearn.tree.DecisionTreeClassifier` com `criterion="entropy"`.

In [4]:
def print_accs_stats(accs):
    mean, std = np.mean(accs), np.std(accs)
    print("Medidas: %s" % accs)
    print("Média: %2.2f%%" % (mean*100))
    print("Desvio Padrão: %2.2f%%" % (std*100))

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
import random 

random.seed(0)

skf = StratifiedKFold(n_splits=10)
dt_accs = []

for train_index, test_index in skf.split(x,y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dt = DecisionTreeClassifier(criterion='entropy', random_state=0)
    dt.fit(x_train, y_train)
    dt_accs.append(dt.score(x_test, y_test))

print("==== Acurácia para Árvore de Decisão ====")
print_accs_stats(dt_accs)

==== Acurácia para Árvore de Decisão ====
Medidas: [0.69, 0.6, 0.7, 0.65, 0.64, 0.69, 0.71, 0.7, 0.73, 0.7]
Média: 68.10%
Desvio Padrão: 3.70%


---
### Questão 03.
Agora utilize 10-fold cross-validation estratificado para estimar a acurácia do classificador `sklearn.naive_bayes.GaussianNB`

In [6]:
from sklearn.naive_bayes import GaussianNB

skf = StratifiedKFold(n_splits=10)
nb_accs = []

for train_index, test_index in skf.split(x,y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    nb = GaussianNB()
    nb.fit(x_train, y_train)
    nb_accs.append(nb.score(x_test, y_test))
    
print("==== Acurácia para Naive Bayes ====")
print_accs_stats(nb_accs)

==== Acurácia para Naive Bayes ====
Medidas: [0.72, 0.64, 0.78, 0.71, 0.77, 0.79, 0.64, 0.8, 0.73, 0.72]
Média: 73.00%
Desvio Padrão: 5.42%


---
### Questão 04.
Repita o procedimento da *Questão 02*, mas agora **calcule e imprima** a matriz de confusão do dataset, bem como a **acurácia calculada através da matriz de confusão** (o valor deve ser o mesmo do calculado anteriormente).

Para isso, inicialize uma matriz $2\times2$ com valores 0 e some, para cada fold, a matriz de confusão no conjunto de teste. Essa matriz irá funcionar basicamente como um acumulador e, ao final das 10 execuções, ela representará a matriz de confusão no conjunto como um todo.

In [7]:
# compute_accuracy calcula valor de acurácia à partir da matriz de confusão
def compute_accuracy(cm):
    return (cm[0,0] + cm[1,1]) / np.sum(cm) 
    

In [11]:
from sklearn.metrics import confusion_matrix

skf = StratifiedKFold(n_splits=10)
dt_cm = np.zeros((2,2), dtype=int)

for train_index, test_index in skf.split(x,y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dt = DecisionTreeClassifier(criterion='entropy', random_state=0)
    dt.fit(x_train, y_train)
    y_pred = dt.predict(x_test)
    dt_cm_fold = confusion_matrix(y_test, y_pred, ['good', 'bad'])
    dt_cm += dt_cm_fold

print("==== Acurácia para Árvore de Decisão - à partir da matriz de confusão ====")
print("Matriz de confusão:\n %s" % dt_cm)
print("Acurácia: %2.2f%%" % (compute_accuracy(dt_cm)*100))

==== Acurácia para Árvore de Decisão - à partir da matriz de confusão ====
Matriz de confusão:
 [[542 158]
 [161 139]]
Acurácia :68.10%


---
### Questão 05.
Repita o procedimento da *Questão 04*, mas ao invés de usar o classificado `sklearn.tree.DecisionTreeClassifier`, utilize `sklearn.naive_bayes.GaussianNB`

In [12]:
skf = StratifiedKFold(n_splits=10)
nb_cm = np.zeros((2,2), dtype=int)

for train_index, test_index in skf.split(x,y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    nb = GaussianNB()
    nb.fit(x_train, y_train)
    y_pred = nb.predict(x_test)
    nb_cm_fold = confusion_matrix(y_test, y_pred, ['good', 'bad'])
    nb_cm += nb_cm_fold

print("==== Acurácia para Naïve Bayes - à partir da matriz de confusão ====")
print("Matriz de confusão:\n %s" % nb_cm)
print("Acurácia: %2.2f%%" % (compute_accuracy(nb_cm)*100))

==== Acurácia para Naïve Bayes - à partir da matriz de confusão ====
Matriz de confusão:
 [[544 156]
 [114 186]]
Acurácia: 73.00%


---

### Questão 06.
Calcule o custo associado a cada classificador e defina qual deles é o melhor. Para isso utilize as matrizes de confusão calculadas nas questões 04 e 05 e a matriz de custo especificada na descrição do dataset.

In [13]:
cost_matrix = np.array([[0,1],[5,0]])
print(cost_matrix)

[[0 1]
 [5 0]]


In [14]:
# Para Árvores de Decisão
dt_cost = np.multiply(cost_matrix, dt_cm)

print("==== Árvore de Decisão ====")

print("Matriz de confusão:\n %s" % dt_cm)
print("Matriz de custo:\n %s" % dt_cost)

print("Custo TOTAL: ", np.sum(dt_cost))

==== Árvore de Decisão ====
Matriz de confusão:
 [[542 158]
 [161 139]]
Matriz de custo:
 [[  0 158]
 [805   0]]
Custo TOTAL:  963


In [15]:
# Para Naïve Bayes
nb_cost = np.multiply(cost_matrix, nb_cm)

print("==== Naïve Bayes ====")

print("Matriz de confusão:\n %s" % nb_cm)
print("Matriz de custo:\n %s" % nb_cost)

print("Custo TOTAL: ", np.sum(nb_cost))

==== Naïve Bayes ====
Matriz de confusão:
 [[544 156]
 [114 186]]
Matriz de custo:
 [[  0 156]
 [570   0]]
Custo TOTAL:  726


É possível observar que a acurácia obtida para o modelo Naïve Bayes (73%) é maior que a obtida para a Árvore de Decisão (68.10%). Além disso, o custo associado ao Naïve Bayes (726) foi menor do que aquele associado à Árvore (963).

Assim, tomando como base o conjunto de dados oferecidos, podemos concluir que *utilizar o modelo Naïve Bayes é mais adequado para este problema.*

---
### Questão 07.
Utilize o software Weka (https://www.cs.waikato.ac.nz/~ml/weka/) e realize classificação utilizando a árvore de decisão  J48. Reporte a acurácia do classificador sem pruning, com pruning=0.25 e pruning=0.1. 

Após o treinamento dos modelos usando o Weka à partir do conjunto de dados fornecidos, obtemos os seguintes valores para a acurácia:

- Sem pruning: **70.4%**
- Com pruning=0.25: **73.9%** 
- Com pruning=0.1: **72.5%**
